In [3]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import numpy as np
import json

#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim


import optuna
from optuna.trial import Trial
import pickle


import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions<
from multimodal.helper_functions import reproducible_comp, get_final_df
from multimodal.model_architecture import MultimodalModel

reproducible_comp()

In [4]:
file_path = "../../../data/processed_data/force_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_inner, category_force_inner, category_names, df_force_inner = get_final_df(data, is_acc_data=0, is_multilabel=1)

file_path = "../../../data/processed_data/acc_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_inner, category_acc_inner, category_names, df_acc_inner = get_final_df(data, is_acc_data=1, is_multilabel=1)

In [5]:
labels_force = category_force_inner.copy()
features_force = features_force_inner.copy()
print("Force Features Shape: ", features_force.shape)
print("Force Labels Shape: ", labels_force.shape)

labels_acc = category_acc_inner.copy()
features_acc = features_acc_inner.copy()
print("Acceleration Features Shape: ", features_acc.shape)
print("Acceleration Labels Shape: ", labels_acc.shape)

Force Features Shape:  (1350, 1)
Force Labels Shape:  (1350, 4)
Acceleration Features Shape:  (1350, 3)
Acceleration Labels Shape:  (1350, 4)


#### For Force

In [6]:
max_length = 150
for i in range (len(features_force)):
    if(len(features_force[i][0]<max_length)):
        features_force[i][0]=np.pad(features_force[i][0], (0,max_length-len(features_force[i][0])), 'constant', constant_values=(0))
feat_force= np.zeros(len(features_force) *max_length).reshape(-1, max_length)
for i in range(len(features_force)):
    feat_force[i]= np.concatenate((features_force[i][0]), axis=None)
    

print("Updated Features Shape: ", feat_force.shape)

Updated Features Shape:  (1350, 150)


#### For Acceleration

In [7]:
max_length = 500

for i in range (len(features_acc)):
    if(len(features_acc[i][0]<max_length)):
        features_acc[i][0]=np.pad(features_acc[i][0], (0,max_length-len(features_acc[i][0])), 'constant', constant_values=(0))
        
    if(len(features_acc[i][1]<max_length)):
        features_acc[i][1]=np.pad(features_acc[i][1], (0,max_length-len(features_acc[i][1])), 'constant', constant_values=(0))
    
    if(len(features_acc[i][2]<max_length)):
        features_acc[i][2]=np.pad(features_acc[i][2], (0,max_length-len(features_acc[i][2])), 'constant', constant_values=(0))

feat_acc = np.zeros(len(features_acc)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features_acc)):
    feat_acc[i]= np.concatenate((features_acc[i][0],features_acc[i][1], features_acc[i][2]), axis=None)
    

print("Updated Features_acc Shape: ", feat_acc.shape)

Updated Features_acc Shape:  (1350, 1500)


In [8]:
X_train_force, X_test_force, y_train_force, y_test_force = train_test_split(feat_force, labels_force, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_force)

X_train_acc, X_test_acc, y_train_acc, y_test_acc = train_test_split(feat_acc, labels_acc, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_acc)                                                   

In [9]:
X_train_force = X_train_force.reshape(-1, 1, feat_force.shape[1])
X_test_force = X_test_force.reshape(-1, 1, feat_force.shape[1])

X_train_acc = X_train_acc.reshape(-1, 1, feat_acc.shape[1])
X_test_acc = X_test_acc.reshape(-1, 1, feat_acc.shape[1])

print(X_train_force.shape)
print(y_train_force.shape)
print(X_test_force.shape)
print(y_test_acc.shape)

print(X_train_acc.shape)
print(y_train_acc.shape)
print(X_test_acc.shape)
print(y_test_acc.shape)

(1215, 1, 150)
(1215, 4)
(135, 1, 150)
(135, 4)
(1215, 1, 1500)
(1215, 4)
(135, 1, 1500)
(135, 4)


In [10]:
from torch.utils.data import DataLoader, TensorDataset

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your multimodal data and labels for both modalities
x_train_force = torch.tensor(X_train_force, dtype=torch.float32)
y_train_force = torch.tensor(y_train_force, dtype=torch.float32)

x_train_acc = torch.tensor(X_train_acc, dtype=torch.float32)
y_train_acc = torch.tensor(y_train_acc, dtype=torch.float32)

# Define the objective function for hyperparameter optimization
def multimodal_objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        force_n_layers = trial.suggest_int("force_n_layers", 1, 7)
        force_kernel_size = trial.suggest_int("force_kernel_size", 3, 9)
        force_stride = trial.suggest_int("force_stride", 1, 7)
        force_padding = trial.suggest_int("force_padding", 1, 10)
        force_hidden_channels = trial.suggest_int("force_hidden_channels", 16, 256)
        force_pooling_type = trial.suggest_categorical("force_pooling_type", ["max", "avg"])
        force_use_batch_norm = trial.suggest_categorical("force_use_batch_norm", [True, False])

        acc_n_layers = trial.suggest_int("acc_n_layers", 1, 7)
        acc_kernel_size = trial.suggest_int("acc_kernel_size", 3, 9)
        acc_stride = trial.suggest_int("acc_stride", 1, 7)
        acc_padding = trial.suggest_int("acc_padding", 1, 10)
        acc_hidden_channels = trial.suggest_int("acc_hidden_channels", 16, 256)
        acc_pooling_type = trial.suggest_categorical("acc_pooling_type", ["max", "avg"])
        acc_use_batch_norm = trial.suggest_categorical("acc_use_batch_norm", [True, False])


        # Split the data into training and validation sets
        x_train_split_force, x_valid_split_force, y_train_split_force, y_valid_split_force = train_test_split(
            x_train_force, y_train_force, test_size=0.2, random_state=42
        )
        x_train_split_acc, x_valid_split_acc, y_train_split_acc, y_valid_split_acc = train_test_split(
            x_train_acc, y_train_acc, test_size=0.2, random_state=42
        )

        # Create datasets and data loaders for both modalities
        train_dataset_force = TensorDataset(x_train_split_force, y_train_split_force)
        valid_dataset_force = TensorDataset(x_valid_split_force, y_valid_split_force)

        train_dataset_acc = TensorDataset(x_train_split_acc, y_train_split_acc)
        valid_dataset_acc = TensorDataset(x_valid_split_acc, y_valid_split_acc)

        train_loader_force = DataLoader(train_dataset_force, batch_size=64, shuffle=True)
        valid_loader_force = DataLoader(valid_dataset_force, batch_size=64, shuffle=False)

        train_loader_acc = DataLoader(train_dataset_acc, batch_size=64, shuffle=True)
        valid_loader_acc = DataLoader(valid_dataset_acc, batch_size=64, shuffle=False)

        # Create the multimodal model with the suggested hyperparameters
        model = MultimodalModel(input_channels, num_classes, 
                 force_n_layers, force_kernel_size, force_stride, force_padding, force_hidden_channels,
                 force_pooling_type, force_use_batch_norm,
                 acc_n_layers, acc_kernel_size, acc_stride, acc_padding, acc_hidden_channels,
                 acc_pooling_type, acc_use_batch_norm, dropout_prob).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for (inputs_acc, labels), (inputs_force,lab) in zip(train_loader_acc, train_loader_force):
                inputs_acc, labels, inputs_force = inputs_acc.to(device), labels.to(device), inputs_force.to(device)

                optimizer.zero_grad()

                outputs = model(inputs_force, inputs_acc)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader_acc)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for (inputs_acc_valid, labels_valid), (inputs_force_valid,lab) in zip(valid_loader_acc, valid_loader_force):
                    inputs_acc_valid, labels_valid, inputs_force_valid = inputs_acc_valid.to(device), labels_valid.to(device), inputs_force_valid.to(device)
                    outputs_valid = model(inputs_force_valid, inputs_acc_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader_acc)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss


In [11]:
# Create an Optuna study for multimodal model hyperparameter optimization
study = optuna.create_study(
    study_name="Case#7: Multi Modal Inner",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 150

# Perform hyperparameter optimization for the multimodal model
study.optimize(multimodal_objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-10 19:39:01,429] A new study created in RDB with name: Case#7: Multi Modal Inner


Epoch 10/50, Training Loss: 0.4622468166053295, Validation Loss: 0.4533188194036484
Epoch 20/50, Training Loss: 0.39117880165576935, Validation Loss: 0.4062911570072174
Epoch 30/50, Training Loss: 0.3710224889218807, Validation Loss: 0.400945283472538
Epoch 40/50, Training Loss: 0.3499736599624157, Validation Loss: 0.38380464166402817


[I 2023-10-10 19:42:16,515] Trial 0 finished with value: 0.37178926169872284 and parameters: {'lr': 7.77964664827174e-05, 'dropout_prob': 0.2943048763629448, 'force_n_layers': 2, 'force_kernel_size': 7, 'force_stride': 6, 'force_padding': 5, 'force_hidden_channels': 222, 'force_pooling_type': 'max', 'force_use_batch_norm': True, 'acc_n_layers': 6, 'acc_kernel_size': 8, 'acc_stride': 1, 'acc_padding': 3, 'acc_hidden_channels': 194, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}. Best is trial 0 with value: 0.37178926169872284.


Epoch 50/50, Training Loss: 0.32390739396214485, Validation Loss: 0.37178926169872284


[I 2023-10-10 19:42:17,827] Trial 1 finished with value: 1.0 and parameters: {'lr': 0.0001321622130856446, 'dropout_prob': 0.05659481279594236, 'force_n_layers': 4, 'force_kernel_size': 4, 'force_stride': 3, 'force_padding': 6, 'force_hidden_channels': 134, 'force_pooling_type': 'avg', 'force_use_batch_norm': True, 'acc_n_layers': 5, 'acc_kernel_size': 8, 'acc_stride': 5, 'acc_padding': 2, 'acc_hidden_channels': 35, 'acc_pooling_type': 'max', 'acc_use_batch_norm': False}. Best is trial 0 with value: 0.37178926169872284.


...

Epoch 50/50, Training Loss: 0.1878152433782816, Validation Loss: 0.2794065475463867
Epoch 10/50, Training Loss: 0.3248620415106416, Validation Loss: 0.3749477490782738
Epoch 20/50, Training Loss: 0.23256719019263983, Validation Loss: 0.3079395666718483
Epoch 30/50, Training Loss: 0.1629894166253507, Validation Loss: 0.29415737837553024
Epoch 40/50, Training Loss: 0.12531196023337543, Validation Loss: 0.2973979115486145


[I 2023-10-10 22:39:44,315] Trial 148 finished with value: 0.3655427470803261 and parameters: {'lr': 0.0005347192551254236, 'dropout_prob': 0.23201973795041267, 'force_n_layers': 4, 'force_kernel_size': 6, 'force_stride': 2, 'force_padding': 6, 'force_hidden_channels': 243, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 5, 'acc_kernel_size': 7, 'acc_stride': 2, 'acc_padding': 3, 'acc_hidden_channels': 115, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': True}. Best is trial 142 with value: 0.25029153004288673.


Epoch 50/50, Training Loss: 0.07078771153464913, Validation Loss: 0.3655427470803261
Epoch 10/50, Training Loss: 0.3420633338391781, Validation Loss: 0.39100754261016846
Epoch 20/50, Training Loss: 0.24779232870787382, Validation Loss: 0.30123738944530487
Epoch 30/50, Training Loss: 0.18681959528476, Validation Loss: 0.25890519469976425
Epoch 40/50, Training Loss: 0.14776082942262292, Validation Loss: 0.2705770581960678


[I 2023-10-10 22:41:07,847] Trial 149 finished with value: 0.309914693236351 and parameters: {'lr': 0.000484453088146792, 'dropout_prob': 0.2651018073191849, 'force_n_layers': 4, 'force_kernel_size': 6, 'force_stride': 2, 'force_padding': 6, 'force_hidden_channels': 249, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 5, 'acc_kernel_size': 7, 'acc_stride': 2, 'acc_padding': 4, 'acc_hidden_channels': 108, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': True}. Best is trial 142 with value: 0.25029153004288673.


Epoch 50/50, Training Loss: 0.11484574154019356, Validation Loss: 0.309914693236351


In [13]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.0003027437119614072, 'dropout_prob': 0.2554954189223444, 'force_n_layers': 4, 'force_kernel_size': 6, 'force_stride': 2, 'force_padding': 6, 'force_hidden_channels': 256, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 5, 'acc_kernel_size': 7, 'acc_stride': 2, 'acc_padding': 4, 'acc_hidden_channels': 114, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': True}
Best final validation loss:
0.25029153004288673


In [14]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/multimodal_inner_best_configs.json"

In [17]:
# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [16]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You